<a href="https://colab.research.google.com/github/ivanshauck/measuring-offensive-baseball-value/blob/main/baseball_offensive_value.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I'll now create my statistic and test it out on 2012 mike trout and 2012 miguel cabrera, in the interest of finding out who the more valuable hitter was. The data for their game logs was downloaded from https://www.baseball-reference.com/ . 

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [63]:
data = pd.read_csv('baseball (2).csv')

First off I am calling the result of plugging in the singles, doubles, triples, home runs, sacrifice flies, walks, and stolen bases into the first regression formula from the modeling notebook: offensive total. Here I will calculate that value and divide it by the number of at bats to create offensive total per at bat (OT/PA).

In [64]:
data['PA'] = data['at_bats'] + data['walks'] + data['sacrifice_flies']
data['OT/PA'] = ((.4595*data['singles'])+(.7426*data['doubles'])+(.9956*data['triples'])+(1.4188*data['home_runs'])+(.2964*data['walks'])+(.0893*data['stolen_bases'])+(.6677*data['sacrifice_flies']))/(data['PA'])

Next up is to take ballpark and year factors into account. Taking ones team or the lineup played in into account is beyond the scope of this project. I suspect one would have to analyze various players changes in performance from one lineup to the next, also taking ballpark and year into account. This can get incredibly messy though as a players performance also varies naturally, without the need of external factors, so that method could produce extremely flawed results. Perhaps a study of how many good pitches a player gets would be more appropriate, this will be something to consider for future reference.

The idea here is we will look at the average offensive total per plate appearance averages for the year, and the ballpark. For the ballpark though I will only be taking the averages from visiting teams though, as the average OT/PA for a ballpark would end up coverging to be closer to the home teams general average, so this is done to reduce bias. I'll also be multiplying this average in national league ballparks by the ratio of average OT/PA in american league history to average OT/PA in national league history, this is done to account for the designated hitter factor in the american league. That is, national league parks should technically have less offensive production in general given that they didn't use the DH in the time period this study was done on. Ultimately the american league ballparks have an OT/PA of about 1.02 that of the national league ballparks, so the difference isn't drastic.  

In [65]:
nl_mean = data.loc[data['h_league']=='NL']['OT/PA'].mean()
al_mean = data.loc[data['h_league']== 'AL']['OT/PA'].mean()


So here we see the ballparks and year columns will be label encoded. The average OT/PA of the year and ballpark will be added in two new columns, and will correspond to the year and ballpark of their respective row.

In [66]:
data_x = data.loc[data['home/away']=='away']
ballpark_means = data_x.groupby('ballpark')['OT/PA'].mean()
year_means = data.groupby('year')['OT/PA'].mean()
data['ball_ot_enc'] = data['ballpark'].map(ballpark_means)
data['year_ot_enc'] = data['year'].map(year_means)
data['factor'] = np.where(data['h_league']=='NL', 1.0236743459388857, 1)
data['ball_ot_enc'] = data['ball_ot_enc']*data['factor']

In [67]:
data.head(20)

,Unnamed: 0,h_league,v_triple_plays,year,team,score,at_bats,hits,doubles,triples,home_runs,rbi,sacrifice_hits,sacrifice_flies,walks,stolen_bases,home/away,singles,ballpark,PA,OT/PA,ball_ot_enc,year_ot_enc,factor
0,110272,AL,0.0,1980,TOR,6,36.0,9.0,1.0,1.0,2.0,5.0,0.0,0.0,5.0,2.0,away,5.0,SEA02,41.0,0.208144,0.180886,0.17267,1.000000
1,110273,NL,0.0,1980,ATL,0,30.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,away,3.0,CIN08,30.0,0.045950,0.182014,0.17267,1.023674
2,110274,AL,0.0,1980,BAL,5,31.0,8.0,2.0,2.0,0.0,4.0,1.0,1.0,1.0,0.0,away,4.0,CHI10,33.0,0.190258,0.169719,0.17267,1.000000
3,110275,AL,0.0,1980,DET,5,33.0,9.0,1.0,2.0,2.0,5.0,0.0,0.0,3.0,1.0,away,4.0,KAN06,36.0,0.232997,0.178732,0.17267,1.000000
4,110276,AL,0.0,1980,BOS,5,35.0,10.0,0.0,0.0,2.0,5.0,0.0,0.0,1.0,0.0,away,8.0,MIL05,36.0,0.189167,0.170335,0.17267,1.000000
5,110277,AL,0.0,1980,MIN,9,48.0,14.0,1.0,1.0,2.0,9.0,1.0,0.0,2.0,0.0,away,10.0,OAK01,50.0,0.195272,0.162798,0.17267,1.000000
6,110278,AL,0.0,1980,NYA,0,40.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,away,4.0,ARL01,41.0,0.052059,0.168656,0.17267,1.000000
7,110279,NL,0.0,1980,LAN,2,29.0,2.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0,away,1.0,HOU02,30.0,0.065303,0.153509,0.17267,1.023674
8,110280,NL,0.0,1980,CHN,2,34.0,7.0,3.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,away,4.0,NYC17,35.0,0.124634,0.167749,0.17267,1.023674
9,110281,NL,0.0,1980,SFN,4,34.0,8.0,2.0,0.0,1.0,4.0,0.0,0.0,2.0,0.0,away,5.0,SAN01,36.0,0.160953,0.171781,0.17267,1.023674


Next we will look at the correlation of the label encoded ballpark column and the label encoded year column and look at their correlation to OT/PA for each game. I will use this to weight the impact that year and ballpark have on the new adjusted statistic I will create.

In [68]:
factor_relevance = data[['OT/PA','ball_ot_enc','year_ot_enc']]
factor_relevance.corr()

,OT/PA,ball_ot_enc,year_ot_enc
OT/PA,1.000000,0.123124,0.102868
ball_ot_enc,0.123124,1.000000,0.149063
year_ot_enc,0.102868,0.149063,1.000000


So we see the label encoded ballpark column has a correlation of about .12 to OT/PA, and the label encoded year column a correlation of about .10. So now what I'll do is the following; for each game I will take the ratio of OT/PA in the game to average OT/PA for the year played in, multiply this by the correlation of the label encoded year to OT/PA, and do the exact same with the average OT/PA of ballpark the game is played in. Next I divide this figure by the sum of the two correlations. And finally I'll multiply this by 100, this is mainly for the eye comfort of the statistic enthusiast, the same thing is done with era+ and ops+. I'll call this stat adjusted offensive total per plate appearance plus (or ADJ OT/PA +).

In [69]:
data['ADJ_OT/PA_+'] = (((.102868*(data['OT/PA']/data['year_ot_enc']))+(.149063*(data['OT/PA']/data['ball_ot_enc'])))/(.102868+.149063))*100



In [70]:
data.head()

,Unnamed: 0,h_league,v_triple_plays,year,team,score,at_bats,hits,doubles,triples,home_runs,rbi,sacrifice_hits,sacrifice_flies,walks,stolen_bases,home/away,singles,ballpark,PA,OT/PA,ball_ot_enc,year_ot_enc,factor,ADJ_OT/PA_+
0,110272,AL,0.0,1980,TOR,6,36.0,9.0,1.0,1.0,2.0,5.0,0.0,0.0,5.0,2.0,away,5.0,SEA02,41.0,0.208144,0.180886,0.17267,1.000000,117.304903
1,110273,NL,0.0,1980,ATL,0,30.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,away,3.0,CIN08,30.0,0.045950,0.182014,0.17267,1.023674,25.803167
2,110274,AL,0.0,1980,BAL,5,31.0,8.0,2.0,2.0,0.0,4.0,1.0,1.0,1.0,0.0,away,4.0,CHI10,33.0,0.190258,0.169719,0.17267,1.000000,111.319412
3,110275,AL,0.0,1980,DET,5,33.0,9.0,1.0,2.0,2.0,5.0,0.0,0.0,3.0,1.0,away,4.0,KAN06,36.0,0.232997,0.178732,0.17267,1.000000,132.229860
4,110276,AL,0.0,1980,BOS,5,35.0,10.0,0.0,0.0,2.0,5.0,0.0,0.0,1.0,0.0,away,8.0,MIL05,36.0,0.189167,0.170335,0.17267,1.000000,110.442662


Now if you want to backtrack to get the adusted outcome total for each row, you simply divide ADJ OT/PA + by 100 then multiply it by the number of plate appearances in the game. To get the total for a season you just sum the adjusted outcome total for each row of a players game logs in a single season, and if you want their ADJ OT/PA + for the whole season you can take that sum and divide it by the number of plate appearances in the whole season.

In [71]:
data['ADJ_OT'] = (data['ADJ_OT/PA_+']/100)*data['PA']

In [72]:
data.head()

,Unnamed: 0,h_league,v_triple_plays,year,team,score,at_bats,hits,doubles,triples,home_runs,rbi,sacrifice_hits,sacrifice_flies,walks,stolen_bases,home/away,singles,ballpark,PA,OT/PA,ball_ot_enc,year_ot_enc,factor,ADJ_OT/PA_+,ADJ_OT
0,110272,AL,0.0,1980,TOR,6,36.0,9.0,1.0,1.0,2.0,5.0,0.0,0.0,5.0,2.0,away,5.0,SEA02,41.0,0.208144,0.180886,0.17267,1.000000,117.304903,48.095010
1,110273,NL,0.0,1980,ATL,0,30.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,away,3.0,CIN08,30.0,0.045950,0.182014,0.17267,1.023674,25.803167,7.740950
2,110274,AL,0.0,1980,BAL,5,31.0,8.0,2.0,2.0,0.0,4.0,1.0,1.0,1.0,0.0,away,4.0,CHI10,33.0,0.190258,0.169719,0.17267,1.000000,111.319412,36.735406
3,110275,AL,0.0,1980,DET,5,33.0,9.0,1.0,2.0,2.0,5.0,0.0,0.0,3.0,1.0,away,4.0,KAN06,36.0,0.232997,0.178732,0.17267,1.000000,132.229860,47.602749
4,110276,AL,0.0,1980,BOS,5,35.0,10.0,0.0,0.0,2.0,5.0,0.0,0.0,1.0,0.0,away,8.0,MIL05,36.0,0.189167,0.170335,0.17267,1.000000,110.442662,39.759358


Next I'll use baseball reference to get the game logs for both mike trout and miguel cabrera in 2012. I'll have to write a few functions to be able to properly get these game logs uploaded into a data frame. I'll also have to convert their ballpark played in value to the same format that's in the data from the game logs, that is in their mlb park ID format. I was able to get a dictionary of these mlb park id's from this link: https://www.seamheads.com/ballparks/about.php. If you scroll down on the page there is a download ballparks database link, click on that, download the document, and the park ID's will be found in an xml file inside the document called parks. Baseball reference lets you copy data from game logs in csv format, so what I did was paste that into the first cell of an excel spreadhseet. I then saved it in csv format, but when I uploaded it into a pandas dataframe there was one column, the first row was the column names separated by commas, then the values were in the following rows, also all in one column separated by commas, that's the main reason for all the formatting you'll see.

In [73]:
def bb_ref_to_pd(csv_file):
  frame = pd.read_csv(csv_file)
  frame_columns = list(frame.iloc[0].values)
  frame_columns1 = frame_columns[0].split(',')
  frame_rows = frame['#NAME?'].str.split(',',expand=True)
  frame_rows = frame_rows.drop([0,len(frame_rows.index)-1])
  frame_rows.columns = frame_columns1
  data = frame_rows.copy()
  return data

cabrera2012 = bb_ref_to_pd('mc2012.csv')
trout2012 = bb_ref_to_pd('mt2012.csv')  



In [75]:
trout2012.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140 entries, 1 to 140
Data columns (total 38 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rk      140 non-null    object
 1   Gcar    140 non-null    object
 2   Gtm     140 non-null    object
 3   Date    140 non-null    object
 4   Tm      140 non-null    object
 5           140 non-null    object
 6   Opp     140 non-null    object
 7   Rslt    140 non-null    object
 8   Inngs   140 non-null    object
 9   PA      140 non-null    object
 10  AB      140 non-null    object
 11  R       140 non-null    object
 12  H       140 non-null    object
 13  2B      140 non-null    object
 14  3B      140 non-null    object
 15  HR      140 non-null    object
 16  RBI     140 non-null    object
 17  BB      140 non-null    object
 18  IBB     140 non-null    object
 19  SO      140 non-null    object
 20  HBP     140 non-null    object
 21  SH      140 non-null    object
 22  SF      140 non-null    ob

Here I create a column called home stadium based on whether or not there is an @ sign before the opponent column.

In [76]:
pd.set_option('display.max_rows',None)
stadiums = trout2012.iloc[:,5:7]
cabrera2012['home_stadium'] = np.where(cabrera2012.iloc[:,5]=='@', cabrera2012['Opp'], 'DET05')
trout2012['home_stadium'] = np.where(trout2012.iloc[:,5]=='@', trout2012['Opp'], 'ANA01')


Here I convert the team names into their mlb park codes, so as to match the park codes from the orginal dataset.

In [77]:

cabrera2012['home_stadium'] = cabrera2012['home_stadium'].replace(['CLE'],'CLE08')
trout2012['home_stadium'] = trout2012['home_stadium'].replace(['CLE'],'CLE08')
cabrera2012['home_stadium'] = cabrera2012['home_stadium'].replace(['MIN'],'MIN04')
trout2012['home_stadium'] = trout2012['home_stadium'].replace(['MIN'],'MIN04')
cabrera2012['home_stadium'] = cabrera2012['home_stadium'].replace(['TEX'],'ARL02')
trout2012['home_stadium'] = trout2012['home_stadium'].replace(['TEX'],'ARL02')
cabrera2012['home_stadium'] = cabrera2012['home_stadium'].replace(['LAA'],'ANA01')
trout2012['home_stadium'] = trout2012['home_stadium'].replace(['SDP'],'SAN02')
cabrera2012['home_stadium'] = cabrera2012['home_stadium'].replace(['OAK'],'OAK01')
trout2012['home_stadium'] = trout2012['home_stadium'].replace(['OAK'],'OAK01')
cabrera2012['home_stadium'] = cabrera2012['home_stadium'].replace(['SEA'],'SEA03')
trout2012['home_stadium'] = trout2012['home_stadium'].replace(['SEA'],'SEA03')
trout2012['home_stadium'] = trout2012['home_stadium'].replace(['COL'],'DEN02')
trout2012['home_stadium'] = trout2012['home_stadium'].replace(['LAD'],'LOS03')
cabrera2012['home_stadium'] = cabrera2012['home_stadium'].replace(['BAL'],'BAL12')
trout2012['home_stadium'] = trout2012['home_stadium'].replace(['BAL'],'BAL12')
cabrera2012['home_stadium'] = cabrera2012['home_stadium'].replace(['TOR'],'TOR02')
trout2012['home_stadium'] = trout2012['home_stadium'].replace(['TOR'],'TOR02')
cabrera2012['home_stadium'] = cabrera2012['home_stadium'].replace(['NYY'],'NYC21')
trout2012['home_stadium'] = trout2012['home_stadium'].replace(['NYY'],'NYC21')
trout2012['home_stadium'] = trout2012['home_stadium'].replace(['DET'],'DET05')
cabrera2012['home_stadium'] = cabrera2012['home_stadium'].replace(['CHW'],'CHI12')
trout2012['home_stadium'] = trout2012['home_stadium'].replace(['CHW'],'CHI12')
cabrera2012['home_stadium'] = cabrera2012['home_stadium'].replace(['BOS'],'BOS07')
trout2012['home_stadium'] = trout2012['home_stadium'].replace(['BOS'],'BOS07')
cabrera2012['home_stadium'] = cabrera2012['home_stadium'].replace(['KCR'],'KAN06')
trout2012['home_stadium'] = trout2012['home_stadium'].replace(['KCR'],'KAN06')
cabrera2012['home_stadium'] = cabrera2012['home_stadium'].replace(['CIN'],'CIN09')
cabrera2012['home_stadium'] = cabrera2012['home_stadium'].replace(['CHC'],'CHI11')
cabrera2012['home_stadium'] = cabrera2012['home_stadium'].replace(['PIT'],'PIT08')
cabrera2012['home_stadium'] = cabrera2012['home_stadium'].replace(['TBR'],'STP01')



In [78]:
cabrera2012.to_csv('cabrera.csv')
trout2012.to_csv('trout.csv')

Here I check the average OT/PA for 2012, I'll be able to just add a column with this figure as every value, to later on calculate the adjusted stat using the year and ballpark factor.

In [79]:
data.loc[data['year']==2012].year_ot_enc

49487     0.174942
49488     0.174942
49489     0.174942
49490     0.174942
49491     0.174942
49492     0.174942
49493     0.174942
49494     0.174942
49495     0.174942
49496     0.174942
49497     0.174942
49498     0.174942
49499     0.174942
49500     0.174942
49501     0.174942
49502     0.174942
49503     0.174942
49504     0.174942
49505     0.174942
49506     0.174942
49507     0.174942
49508     0.174942
49509     0.174942
49510     0.174942
49511     0.174942
49512     0.174942
49513     0.174942
49514     0.174942
49515     0.174942
49516     0.174942
49517     0.174942
49518     0.174942
49519     0.174942
49520     0.174942
49521     0.174942
49522     0.174942
49523     0.174942
49524     0.174942
49525     0.174942
49526     0.174942
49527     0.174942
49528     0.174942
49529     0.174942
49530     0.174942
49531     0.174942
49532     0.174942
49533     0.174942
49534     0.174942
49535     0.174942
49536     0.174942
49537     0.174942
49538     0.174942
49539     0.

Here I'll take the mean OT/PA per ballpark for visiting teams from the original game log data and map it to this dataset.

In [80]:
cabrera2012['ball_ot_enc'] = cabrera2012['home_stadium'].map(ballpark_means)
trout2012['ball_ot_enc'] = trout2012['home_stadium'].map(ballpark_means)
cabrera2012['year_ot_enc'] = .174942
trout2012['year_ot_enc'] = .174942

Trout had a missing value in one of the rows in the sac flies column, I'll just replace this with 0. He didn't play that game.

In [81]:
trout2012.at[58, 'SF'] = 0

I convert all the offensive outcomes plus plate appearance columns from object data type to integer data type.

In [82]:
cabrera2012['H'] = cabrera2012['H'].astype(int)
cabrera2012['2B'] = cabrera2012['2B'].astype(int)
cabrera2012['3B'] = cabrera2012['3B'].astype(int)
cabrera2012['HR'] = cabrera2012['HR'].astype(int)
cabrera2012['SB'] = cabrera2012['SB'].astype(int)
cabrera2012['SF'] = cabrera2012['SF'].astype(int)
cabrera2012['BB'] = cabrera2012['BB'].astype(int)
cabrera2012['PA'] = cabrera2012['PA'].astype(int)
trout2012['H'] = trout2012['H'].astype(int)
trout2012['2B'] = trout2012['2B'].astype(int)
trout2012['3B'] = trout2012['3B'].astype(int)
trout2012['HR'] = trout2012['HR'].astype(int)
trout2012['BB'] = trout2012['BB'].astype(int)
trout2012['SF'] = trout2012['SF'].astype(int)
trout2012['PA'] = trout2012['PA'].astype(int)
trout2012['SB'] = trout2012['SB'].astype(int)

I write a function called stat creator that takes the values from the baseball reference seasonal game logs for each player, and outputs the offensive total, offensive total per plate appearance, and adjusted offensive total per plate appearance plus for each game.

In [83]:
def stat_creator(frame):
  frame['1B'] = frame['H'] - frame['2B'] - frame['3B'] - frame['HR']
  frame['OT'] = (.4595*frame['1B']+.7426*frame['2B']+.9956*frame['3B']+1.4188*frame['HR']+.2964*frame['BB']+.0893*frame['SB']+.6677*frame['SF'])
  frame['OT/PA'] = frame['OT']/frame['PA']
  frame['ADJ_OT/PA+'] = (((.102868*(frame['OT/PA']/frame['year_ot_enc']))+(.149063*(frame['OT/PA']/frame['ball_ot_enc'])))/(.102868+.149063))*100

stat_creator(trout2012)
stat_creator(cabrera2012)

In [84]:
trout2012.head()

,Rk,Gcar,Gtm,Date,Tm,,Opp,Rslt,Inngs,PA,AB,R,H,2B,3B,HR,RBI,BB,IBB,SO,HBP,SH,SF,ROE,GDP,SB,CS,BA,OBP,SLG,OPS,BOP,aLI,WPA,acLI,cWPA,RE24,Pos,home_stadium,ball_ot_enc,year_ot_enc,1B,OT,OT/PA,ADJ_OT/PA+
1,1,41,21 (20),Apr 28,LAA,@,CLE,W2-1,CG,4,4,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,.000,.000,.000,.000,1,.83,-0.082,.47,-0.03%,-0.81,LF,CLE08,0.174640,0.174942,0,0.0000,0.00000,0.000000
2,2,42,22,Apr 29,LAA,@,CLE,L0-4,CG,4,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,.000,.125,.000,.125,1,1.41,-0.021,.87,-0.01%,-0.39,CF,CLE08,0.174640,0.174942,0,0.2964,0.07410,42.400202
3,3,43,23,Apr 30,LAA,,MIN,W4-3,CG,4,4,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,.091,.167,.182,.348,1,.52,0.025,.30,0.01%,0.02,CF,ANA01,0.171475,0.174942,0,0.7426,0.18565,107.390455
4,4,44,24,May 1,LAA,,MIN,W4-0,CG,4,3,1,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,.143,.250,.214,.464,1,.49,0.027,.31,0.01%,0.85,CF,ANA01,0.171475,0.174942,1,0.8452,0.21130,122.227865
5,5,45,26 (1),May 3,LAA,,TOR,L0-5,CG,4,4,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,.222,.300,.278,.578,1,.57,0.066,.40,0.03%,0.22,CF,ANA01,0.171475,0.174942,2,0.9190,0.22975,132.900388


In [85]:
cabrera2012.head()

,Rk,Gcar,Gtm,Date,Tm,,Opp,Rslt,Inngs,PA,AB,R,H,2B,3B,HR,RBI,BB,IBB,SO,HBP,SH,SF,ROE,GDP,SB,CS,BA,OBP,SLG,OPS,BOP,aLI,WPA,acLI,cWPA,RE24,Pos,home_stadium,ball_ot_enc,year_ot_enc,1B,OT,OT/PA,ADJ_OT/PA+
1,1,1352,1,Apr 5,DET,,BOS,W3-2,CG,4,1,0,0,0,0,0,0,3,1,0,0,0,0,0,0,0,0,.000,.750,.000,.750,3,.79,0.061,.72,0.03%,0.67,3B,DET05,0.17789,0.174942,0,0.8892,0.222300,125.824619
2,2,1353,2,Apr 7,DET,,BOS,W10-0,CG,5,5,2,2,0,0,2,3,0,0,1,0,0,0,0,0,0,0,.333,.556,1.333,1.889,3,.43,0.161,.44,0.10%,2.05,3B,DET05,0.17789,0.174942,0,2.8376,0.567520,321.223517
3,3,1354,3,Apr 8,DET,,BOS,W13-12,CG(11),6,5,3,3,0,0,1,5,0,0,0,0,0,1,0,1,0,0,.455,.533,1.273,1.806,3,2.25,0.480,2.37,0.30%,2.15,3B,DET05,0.17789,0.174942,2,3.0055,0.500917,283.525186
4,4,1355,4,Apr 10,DET,,TBR,W5-2,CG,4,2,1,1,1,0,0,0,2,1,0,0,0,0,0,0,0,0,.462,.579,1.231,1.810,3,1.48,0.134,1.60,0.09%,0.70,3B,DET05,0.17789,0.174942,0,1.3354,0.333850,188.963334
5,5,1356,5,Apr 11,DET,,TBR,L2-4,CG,4,4,0,2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,.471,.565,1.059,1.624,3,1.08,0.055,1.13,0.04%,0.82,3B,DET05,0.17789,0.174942,2,0.9190,0.229750,130.041414


Here I create an adjusted OT stat per each game.

In [86]:
trout2012['ADJ_OT'] = (trout2012['ADJ_OT/PA+']/100)*trout2012['PA']
cabrera2012['ADJ_OT'] = (cabrera2012['ADJ_OT/PA+']/100)*cabrera2012['PA']



The row indexed at 58 for the trout game logs is a problem for the next calculations, and he didn't even play that game so I drop it. Then I also drop Cabrera's last row since it wasn't a game log.

In [87]:
trout2012 = trout2012.drop([140,58])
cabrera2012 = cabrera2012.drop(162)

Here I finish up, the total adjusted offensive total stat is arrived at through summing up the adjusted offensive total for each game. Then the final stat is their adjusted offensive total per plate appearance plus, which is arrived at by taking their adjusted offensive total for the whole season, dividing it by the total number of plate apperances in that season, then muliplying that figure by 100.

In [88]:
print('adjusted offensive total for trout in 2012: {}'.format(sum(trout2012['ADJ_OT'])))
print('adjusted offensive total for cabrera in 2012: {}'.format(sum(cabrera2012['ADJ_OT'])))
print('ADJ OT/PA + for trout in 2012: {}'.format((sum(trout2012['ADJ_OT'])/(sum(trout2012['PA'])))*100))
print('ADJ OT/PA + for cabrera in 2012: {}'.format((sum(cabrera2012['ADJ_OT'])/(sum(cabrera2012['PA'])))*100))

adjusted offensive total for trout in 2012: 880.6986634073412
adjusted offensive total for cabrera in 2012: 972.7308901373867
ADJ OT/PA + for trout in 2012: 137.8245169651551
ADJ OT/PA + for cabrera in 2012: 139.55966859933812


It looks like cabrera was more valuable than trout offensively based on this statistic, both for total production and production per plate appearance. Given that he won the triple crown this doesn't exactly come as a shock. Based on the ADJ PA/OT+ stat trout was about 1.38 times more valuable per plate apperance than league average and cabrera about 1.4 times more valuable.

This statistic, unlike OPS+, is measured based on the relative value of outcomes in relation to runs scored. This is why the ADJ OT/PA + figure may not be high as some OPS+ numbers may be. OPS+ rates the value of base hits solely based on how many bases the player collects, so more extra base hits will give you a higher figure ultimately than ADJ PA/OT +. In an extremely competitive professional league being almost one and a half times better than the average should be considered amazing though. 

This statistic has a solid basis in regression analysis in my opinion, I believe the regression equation shed good light on the relative value of these offensive outcomes. Next up would be to also look at the distributions for negative offensive outcomes, such as strikeouts, groundouts, flyouts, and grounded into double play. Perhaps adding these to the regression equation would allow us to capture more of the variation in the runs scored variable through the variation in the features. I also plan to eventually write a function that will take any players game logs from baseball reference for a specific season, and output their adjusted offensive total and adjusted offensive total per plate appearance plus for that season.